Removing Warning

In [1]:
import warnings
warnings.filterwarnings('ignore')

Loading Panda

In [3]:
import pandas as P

Opening Dataset in python

In [4]:
D = P.read_csv("Nigeria Crash Data U.csv")

Watching Columns

In [5]:
print(D.columns)

Index(['Unnamed: 0', 'Quarter', 'State', 'Total_Crashes', 'Num_Injured',
       'Num_Killed', 'Total_Vehicles_Involved', 'SPV', 'DAD', 'PWR', 'FTQ',
       'Other_Factors'],
      dtype='object')


X = Input Features , Y = Output Features

In [6]:
X = D.drop("Num_Killed" , axis = 1)
Y = D["Num_Killed"]

Train Test Split

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
X_Train , X_Test , Y_Train , Y_Test = train_test_split(X , Y , test_size = 0.2 , random_state = 42)

Applying Model With Hyperparameter tuning (Extra Trees Regressor → Combines many random decision trees to make numeric predictions, trying to reduce overfitting)

In [9]:
from sklearn.ensemble import ExtraTreesRegressor

In [10]:
from sklearn.model_selection import GridSearchCV

In [171]:
PG = {
   "n_estimators": [350],
    "max_depth": [9],
    "min_samples_split": [6],
    "min_samples_leaf": [3],
    "max_features": ['sqrt']
}

In [172]:
G = GridSearchCV(ExtraTreesRegressor(random_state = 42) , PG , cv = 5)

In [173]:
G.fit(X_Train , Y_Train)

GridSearchCV(cv=5, estimator=ExtraTreesRegressor(random_state=42),
             param_grid={'max_depth': [9], 'max_features': ['sqrt'],
                         'min_samples_leaf': [3], 'min_samples_split': [6],
                         'n_estimators': [350]})

In [174]:
BP = G.best_params_

Prediction

In [175]:
YP = G.predict(X_Test)

Performance

In [176]:
import numpy as N
from sklearn.metrics import mean_squared_error , mean_absolute_error , r2_score

In [177]:
R2 = r2_score(Y_Test , YP)
MSE = mean_squared_error(Y_Test , YP)
MAE = mean_absolute_error(Y_Test , YP)
RMSE = N.sqrt(MSE)

In [178]:
print("R2 Score : " , R2 * 100 , "%")
print("\n")
print("MSE : " , MSE)
print("\n")
print("MAE : " , MAE)
print("\n")
print("RMSE : " , RMSE)

R2 Score :  74.1443108135314 %


MSE :  282.5237031769016


MAE :  12.774451400628326


RMSE :  16.808441426167438


In [179]:
Train = G.score(X_Train , Y_Train)
print("Train Score : " , Train * 100 , "%")
Test = G.score(X_Test , Y_Test)
print("Test Score : " , Test * 100 , "%")

Train Score :  80.3113116439249 %
Test Score :  74.1443108135314 %
